In [ ]:
import os, sys
from PyQt5.QtWidgets import QApplication
import vtk
from vtk import vtkCommand
from vtk.qt.QVTKRenderWindowInteractor import QVTKRenderWindowInteractor
import numpy as np

# Framework

This is the prototype framework for our app. There are 2 higher level components, Panes and Viewer. Clicking on the screen will forward to event to the correct Pane. The Pane can then be responsible for dealing with events.

# Pane

In [ ]:
class Pane():
    
    scene = {}
    
    def __init__(self, uid, renderer, viewPort, viewer):
        self.uid = uid
        self.renderer = renderer
        self.viewer = viewer
        self.viewPort = viewPort
        # set viewport
        renderer.SetViewport(viewPort)
        # add to viewer render window
        viewer.renderWindow.AddRenderer(renderer)
    
    def isPointInPane(self, x, y):
        (w, h) = self.viewer.renderWindow.GetSize()
        x = x / w
        y = y / h
        return x > self.viewPort[0] and x < self.viewPort[2] and y > self.viewPort[1] and y < self.viewPort[3]
        
    def onInteractorEvent(self, event, eventData):
        print(self.uid, event, eventData)


# Viewer Interactor Style

In [ ]:
class ViewerInteractorStyle(vtk.vtkInteractorStyleUser):
    
    def __init__(self, viewer):
        self.viewer = viewer        
        self.AddObserver(vtkCommand.LeftButtonPressEvent, self.leftButtonPressEvent)
  
    def leftButtonPressEvent(self, obj, event):    
        (x, y) = obj.GetInteractor().GetEventPosition()
        ratio = self.viewer.devicePixelRatio() # VTK BUG for Retina Display
        x = x*ratio
        y = y*ratio
        
        # forward events to the right panes
        for uid, pane in self.viewer.panes.items():    
            if pane.isPointInPane(x, y):
                pane.onInteractorEvent(event, (x, y))


# Viewer

In [ ]:
class Viewer(QVTKRenderWindowInteractor):
    
    panes = {}
    
    def __init__(self, parent=None):
        super().__init__(parent)
        self.renderWindow =  self.GetRenderWindow()
        self.interactor = self.renderWindow.GetInteractor()
        self.interactor.SetInteractorStyle(ViewerInteractorStyle(self))
    
    def addPane(self, uid, renderer, viewPort):
        self.panes[uid] = Pane(uid, renderer, viewPort, self)
    
    def startEventLoop(self):
        self.interactor.Initialize()
        

In [ ]:
app = QApplication([])

# viewer
viewer = Viewer()
# viewer.setMinimumWidth(800)
# viewer.setMinimumHeight(600)

# Create source
source = vtk.vtkSphereSource()
source.SetCenter(0, 0, 0)
source.SetRadius(5.0)
# Create a mapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(source.GetOutputPort())
# Create an actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)
# Create renderer
ren = vtk.vtkRenderer()
ren.AddActor(actor)

# add panes
viewer.addPane('TL', ren, (0,.5,.5,1))
viewer.addPane('TR', ren, (.5,.5,1,1))
viewer.addPane('BL', ren, (0,0,.5,.5))
viewer.addPane('BR', ren, (.5,0,1,.5))

viewer.show()
viewer.startEventLoop()
sys.exit(app.exec_())